Lyric Generation: 3 Ways (XLNET, GPT-2, LSTM with Pytorch)

Collaborators: Aleks Sekulovski + Grayson Taylor

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
# installs
!pip install flask
!pip install spotipy
!pip install transformers
!pip install sentencepiece==0.1.91
!pip install gpt-2-simple
!nvidia-smi
!pip install transformers
!pip install datasets
!pip install huggingface-hub==0.7
!pip install torch
!pip install wandb
!pip install lyricsgenius
!pip install aiohttp
!pip install langdetect
!pip install accelerate
!pip install --upgrade jax jaxlib 
!pip install --upgrade git+https://github.com/google/flax.git
!pip install tqdm --upgrade
!pip install hf-lfs
!pip install nest_asyncio

# imports
from transformers import XLNetConfig, XLNetModel, XLNetLMHeadModel, XLNetTokenizer
import torch
import lyricsgenius
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from bs4 import BeautifulSoup
import requests
from IPython.display import display, HTML, Javascript, clear_output
import lyricsgenius
from tqdm.notebook import tqdm as bar
import requests
from bs4 import BeautifulSoup
import re
from datasets import Dataset, DatasetDict
import numpy as np
import time
import os
import json
import langdetect
import datetime
from pathlib import Path
import wandb
import pathlib
import nest_asyncio
nest_asyncio.apply()
import asyncio
from concurrent.futures import ProcessPoolExecutor
import sys
sys.setrecursionlimit(99999)
import aiohttp
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
import urllib.request
import urllib.parse
import json
import flask
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import gpt_2_simple as gpt2
import torch.nn as nn
import torch.nn.functional as Functional
import numpy as np
import string
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached huggingface_hub-0.11.1-py3-none-any.whl (182 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.7.0
    Uninstalling huggingface-hub-0.7.0:
      Successfully uninstalled huggingface-hub-0.7.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Tue Dec 13 22:21:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    51W / 400W |    632MiB / 40536MiB |      0%      Default |
|                               |                 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/google/flax.git to /tmp/pip-req-build-yyiqv59g
  Running command git clone -q https://github.com/google/flax.git /tmp/pip-req-build-yyiqv59g
Looking in indexes: https://pypi.org/s

Approach 1: LSTM Model (Pytorch)

In [4]:
### Credits: https://www.analyticsvidhya.com/blog/2020/08/build-a-natural-language-generation-nlg-system-using-pytorch/

# clean alphabet for word generation
def clean_lyric(lyric):
    return re.sub("[^a-z' ]", "", lyric).replace("'", "")

# create and return list of all possible sequences
def create_sequences(lyric, seq_len):
    sequences = []
    if len(lyric.split()) <= seq_len:
        return [lyric]
    for itr in range(seq_len, len(lyric.split())):
        curr_seq = lyric.split()[itr - seq_len:itr + 1]
        sequences.append(" ".join(curr_seq))
    return sequences

# gets numbers for words in input sequence
def get_seq_idx(sequence):
    return [word_to_idx[word] for word in sequence.split()]

# gets the next batch
def get_next_batch(x, y, batch_size):
    for itr in range(batch_size, x.shape[0], batch_size):
        batch_x = x[itr - batch_size:itr, :]
        batch_y = y[itr - batch_size:itr, :]
        yield batch_x, batch_y

# make inputs, detach hidden state, get model output, get token Ps and reshape
# get top 3 values, select 1 and return it (along with the hidden state)
def predict(model, token, hidden_layer):
    x = np.array([[word_to_idx[token]]])
    inputs = torch.from_numpy(x).type(torch.LongTensor)
    hidden = tuple([layer.data for layer in hidden_layer])
    out, hidden = model(inputs, hidden)
    prob = Functional.softmax(out, dim=1).data.numpy()
    prob = prob.reshape(prob.shape[1],)
    top_tokens = prob.argsort()[-3:][::-1]
    selected_index = top_tokens[0]
    return idx_to_word[selected_index], hidden

# creates the string
def generate(model, num_words, start_text):
    model.eval()
    hidden = model.init_hidden(1)
    tokens = start_text.split()
    for token in start_text.split():
        curr_token, hidden = predict(model, token, hidden)
    tokens.append(curr_token)
    for token_num in range(num_words - 1):
        token, hidden = predict(model, tokens[-1], hidden)
        tokens.append(token)
    return " ".join(tokens)

# calls generate to create output string
def get_lyric(start_text, num_words):
    return generate(model, num_words, start_text.lower())

In [5]:
# the meaty part (LSTM)
class LSTM(nn.Module):
    # initialize variables
    def __init__(self, num_hidden, num_layers, embed_size, drop_prob, lr):
        #print("Initializing model...")
        super().__init__()
        self.drop_prob = drop_prob
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.lr = lr
        # define embedded layer, LSTM, dropout layer, andd fully connected layer
        self.embedded = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, num_hidden, num_layers, dropout = drop_prob, batch_first = True)
        self.dropout = nn.Dropout(drop_prob) 
        self.fc = nn.Linear(num_hidden, vocab_size)      
    
    # forward propagate
    def forward(self, x, hidden_layer):      
        # pass input through embedding layer
        embedded = self.embedded(x)        
        # get outputs and hidden layer from LSTM layer
        lstm_output, hidden_layer = self.lstm(embedded, hidden_layer)        
        # pass through a dropout layer and reshape
        dropout_out = self.dropout(lstm_output).reshape(-1, self.num_hidden) 
        # put "out" through the fully-connected layer
        out = self.fc(dropout_out)
        # return final output and hidden layer
        return out, hidden_layer
    
    # init hidden layer
    def init_hidden(self, batch_size):
        # Create a weight torch using the parameters of the model
        weight = next(self.parameters()).data
        # initialize the hidden layer using the weight torch
        hidden_layer = (weight.new(self.num_layers, batch_size, self.num_hidden).zero_(), weight.new(self.num_layers, batch_size, self.num_hidden).zero_())
        # return the hidden layer
        return hidden_layer

In [31]:
file = open("/content/drive/MyDrive/adele.txt", "r", encoding = "utf8")
text = file.read()
text = text.replace("\n\n", "\n")

lyrics = text.lower().split("\n")
lyrics = np.unique(lyrics)[1:].tolist()

cleaned_lyrics = [clean_lyric(lyric) for lyric in lyrics]

seq_size = 5

# get every sequence
raw_sequences = [create_sequences(lyric, seq_size) for lyric in cleaned_lyrics]

# unique sentences only
sequences = np.unique(np.array(sum(raw_sequences, []))).tolist()
print(sequences)

uniq_words = np.unique(np.array(" ".join(sequences).split(" ")))
uniq_words_idx = np.arange(uniq_words.size)

word_to_idx = dict(zip(uniq_words.tolist(), uniq_words_idx.tolist()))
idx_to_word = dict(zip(uniq_words_idx.tolist(), uniq_words.tolist()))

vocab_size = len(word_to_idx)

# intialize empty lists
x_word = []
y_word = []

# iterate through every sequence
for seq in sequences:
    if (len(seq.split()) != seq_size + 1):
        continue
    # add words to sequences
    x_word.append(" ".join(seq.split()[:-1]))
    y_word.append(" ".join(seq.split()[1:]))

x_idx = np.array([get_seq_idx(word) for word in x_word])
y_idx = np.array([get_seq_idx(word) for word in y_word])

# initialize parameters for LSTM model
num_hidden = 256
num_layers = 4
embed_size = 200
drop_prob = 0.3
lr = 0.001
num_epochs = 30
batch_size = 30

# create LSTM model, choosing optimizer and loss function, and training model
model = LSTM(num_hidden, num_layers, embed_size, drop_prob, lr)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)
loss_func = nn.CrossEntropyLoss()
model.train()

for epoch in range(num_epochs):
    # initialize hidden state
    hidden_layer = model.init_hidden(batch_size)
        
    for x, y in get_next_batch(x_idx, y_idx, batch_size):            
        # numpy -> Pytorch
        inputs = torch.from_numpy(x).type(torch.LongTensor)
        act = torch.from_numpy(y).type(torch.LongTensor)
        # reformat hidden layer
        hidden_layer = tuple([layer.data for layer in hidden_layer])
        # get zero-accumulated gradients from the model
        model.zero_grad()            
        # get output
        output, hidden = model(inputs, hidden_layer)           
        # calculate loss from this prediction
        loss = loss_func(output, act.view(-1))
        # backpropagation
        loss.backward()
        # prevents exploding gradient problem
        nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()    

get_lyric("love", 8)

['a fuck so hes probably just', 'a fuckin shrink sheesh i already', 'a fuckin walkin paradox no im', 'a motherfuckin goblin', 'about five seven of his bitches', 'about i start a team of', 'ace gon put that fuckin hole', 'actions speak louder than words let', 'after bowling i went home for', 'aint no v tech shit or', 'airplane that that faggot nigga bob', 'all i want fuck money diamonds', 'all your problems hes fuckin awesome', 'an overachiever so how about i', 'and danced around the house in', 'and hes not fuckin workin i', 'and im wolf that was me', 'and pick up stevie wonder to', 'and stab bruno mars in his', 'and tina still aint perm her', 'and wont stop until the cops', 'any bloggin faggot hipster with a', 'around the house in allover print', 'as im mockin deaf rock stars', 'at i got somethin to feed', 'beat deshay up with the stack', 'bedrock harder than a motherfuckin flintstone', 'bitchin this isnt a fuckin hotline', 'black kids never wanted to read', 'bloggin faggot hipster wit

'goblin the in my my em would time wang wang wang wang wang'

Approach 2: XLNET

In [37]:
test_input = open("/content/drive/MyDrive/teardrops on my guitar.txt", "r", encoding = "utf8")
test_input = test_input.read()
test_input = test_input.replace("\n\n", "\n")
test_input = test_input.replace("\n\n", "\n")
#print(test_input)

In [38]:
# Credits: https://huggingface.co/docs/transformers/model_doc/xlnet; https://towardsdatascience.com/build-a-bidirectional-text-generator-with-xlnet-49d9d37b48a9

# Initializing a XLNet configuration
configuration = XLNetConfig()

tokenizer = XLNetTokenizer.from_pretrained('xlnet-large-cased')
model = XLNetLMHeadModel.from_pretrained('xlnet-large-cased')

# Padding text to help Transformer-XL and XLNet with short prompts
# in https://github.com/rusiaaman/XLNet-gen#methodology
# and https://medium.com/@amanrusia/xlnet-speaks-comparison-to-gpt-2-ea1a4e9ba39e
PADDING_TEXT = test_input

torch.manual_seed(0)
# We show how to setup inputs to predict a next token using a bi-directional context.
# We will predict masked tokens
input_ids = torch.tensor(tokenizer.encode(PADDING_TEXT + "I gave you three apples. <mask> have <mask> apples in hands", add_special_tokens=False)).unsqueeze(0)  

targets = [ -6, -4]

perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
perm_mask[0, :, targets] = 1.0  # Previous tokens don't see last token

target_mapping = torch.zeros((1, len(targets), input_ids.shape[1]), dtype=torch.float)  

target_mapping[0, 0, targets[0]] = 1.0  # Our first  prediction 
target_mapping[0, 1, targets[1]] = 1.0  # Our second  prediction 

input_ids_tensor = input_ids.to("cuda")
target_mapping_tensor = target_mapping.to("cuda")
perm_mask_tensor = perm_mask.to("cuda")

model.eval()
if torch.cuda.is_available(): model.to('cuda') #if we have a GPU 

with torch.no_grad():
  outputs = model(input_ids_tensor, perm_mask=perm_mask_tensor, target_mapping=target_mapping_tensor)
next_token_logits = outputs[0]  # Output has shape [target_mapping.size(0), target_mapping.size(1), config.vocab_size]

for j in range(len(targets)):
  predicted_k_indexes = torch.topk(outputs[0][0][j],k=5)
  predicted_logits_list = predicted_k_indexes[0] 
  predicted_indexes_list = predicted_k_indexes[1] 
    
  print ("predicted word:",tokenizer.decode(input_ids[0][targets[j]].item()), j)
  for i,item  in enumerate(predicted_indexes_list):
      the_index = predicted_indexes_list[i].item()
      print("word and logits",tokenizer.decode(the_index),predicted_logits_list[i].item())

predicted word: <mask> 0
word and logits I -17.119409561157227
word and logits You -18.564960479736328
word and logits I -20.358469009399414
word and logits We -20.713825225830078
word and logits You -21.69858741760254
predicted word: <mask> 1
word and logits three -19.076858520507812
word and logits the -21.324016571044922
word and logits two -22.618837356567383
word and logits your -22.90503692626953
word and logits my -23.162578582763672


In [41]:
# Function to select topK tokens from the probability list and 
# then based on the selected K word distribution get sample of random token IDs
def choose_from_top(probs, k=5, sample_size=1):
    ind = np.argpartition(probs, -k)[-k:]
    top_prob = probs[ind]
    # print(tokenizer.decode(ind))
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(k, sample_size, p = top_prob, replace=False)
    token_ids = ind[choice]
    return token_ids

sent = "So I drive home alone"
topk = 10
n = 20
# Lower temperatures make the model more confident in its top choices, while temperatures greater than 1 decrease confidence.
temperature = 5

model.eval()
if torch.cuda.is_available(): model.to('cuda') #if we have a GPU 

sent_tokens = tokenizer.encode(sent, add_special_tokens=False)
mask_tokens = tokenizer.encode('<mask>', add_special_tokens=False)
padding_tokens = tokenizer.encode(PADDING_TEXT, add_special_tokens=False)
   
for i in range(n):
  input = mask_tokens + sent_tokens + mask_tokens     
  target_id1 = -len(input)
  target_id2 = -1

  input_ids = torch.tensor(padding_tokens + input).unsqueeze(0)   # We will predict masked tokens

  perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
  perm_mask[0, :, [target_id1, target_id2]] = 1.0  # Previous tokens don't see last token

  target_mapping = torch.zeros((1, 2, input_ids.shape[1]), dtype=torch.float)  
  target_mapping[0, 0, target_id1] = 1.0  # Our first  prediction 
  target_mapping[0, 1, target_id2] = 1.0  # Our second  prediction 

  input_ids_tensor = input_ids.to("cuda")
  target_mapping_tensor = target_mapping.to("cuda")
  perm_mask_tensor = perm_mask.to("cuda")

  with torch.no_grad():
    outputs = model(input_ids_tensor, perm_mask=perm_mask_tensor, target_mapping=target_mapping_tensor)

  predicted_tokens = []
  
  for j in range(2):
    probs = torch.nn.functional.softmax(outputs[0][0][j]/temperature, dim = 0).to('cpu').numpy()
    predicted_tokens.append(choose_from_top(probs, k=topk, sample_size=1))

  if i % 2 == 0:    
    tok = predicted_tokens[0][0]
    sent_tokens = [tok] + sent_tokens 
    print('left: ', tokenizer.decode(sent_tokens))
  else:     
    tok = predicted_tokens[1][0]
    sent_tokens = sent_tokens + [tok]
    print("right: ", tokenizer.decode(sent_tokens))

left:  it So I drive home alone
right:  it So I drive home alone
left:  through it So I drive home alone
right:  through it So I drive home alone -
left:  my through it So I drive home alone -
right:  my through it So I drive home alone - a
left:  I my through it So I drive home alone - a
right:  I my through it So I drive home alone - a song
left:  That I my through it So I drive home alone - a song
right:  That I my through it So I drive home alone - a song of
left:  See That I my through it So I drive home alone - a song of
right:  See That I my through it So I drive home alone - a song of hope
left:  me See That I my through it So I drive home alone - a song of hope
right:  me See That I my through it So I drive home alone - a song of hope A
left:  Behind me See That I my through it So I drive home alone - a song of hope A
right:  Behind me See That I my through it So I drive home alone - a song of hope A cry
left:  Looking Behind me See That I my through it So I drive home alone -

In [28]:
# create a combination of beam and top-k generation to generate sequences of n tokens from both sides 

import random

padding_tokens = tokenizer.encode(PADDING_TEXT, add_special_tokens=False)
mask_tokens = tokenizer.encode('<mask>', add_special_tokens=False)

model.eval()
if torch.cuda.is_available(): model.to('cuda') #if we have a GPU 

def candidates_gen(sent_tokens, candidate=([], 1, []), d='left', n_candidates=5, topk=20, temperature=5):
  branch_candidates = []  
  cand_tokens = candidate[0]
  
  if d == 'right':    
    input = sent_tokens + cand_tokens + mask_tokens     
    
    target_id = -1
    input_ids = torch.tensor(padding_tokens + input).unsqueeze(0)  

    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[0, :, target_id] = 1.0  # Previous tokens don't see last token
  else:        
    input = mask_tokens + cand_tokens + sent_tokens    
    
    target_id = -len(input)  
    input_ids = torch.tensor(padding_tokens + input).unsqueeze(0)  

    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[0, :, [target_id - i for i in range(100)]] = 1.0  # Mask additional previos tokens to improve left-side generation

  # We will predict masked tokens 
  target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  
  target_mapping[0, 0, target_id] = 1.0  # Our right  prediction 

  if torch.cuda.is_available():
    input_ids_tensor = input_ids.to("cuda")
    target_mapping_tensor = target_mapping.to("cuda")
    perm_mask_tensor = perm_mask.to("cuda")
  else:
    input_ids_tensor = input_ids
    target_mapping_tensor = target_mapping
    perm_mask_tensor = perm_mask

  with torch.no_grad():
    outputs = model(input_ids_tensor, perm_mask=perm_mask_tensor, target_mapping=target_mapping_tensor)

  probs = torch.nn.functional.softmax(outputs[0][0][0]/temperature, dim = 0)
  selected_indexes = choose_from_top(probs.to('cpu').numpy(), k=topk, sample_size=n_candidates)
  selected_probs = probs[selected_indexes]

  for i,item  in enumerate(selected_indexes):
      the_index = item.item()
      if d == "right":
        new_sent = cand_tokens + [the_index]
      elif d == "left":
        new_sent = [the_index] + cand_tokens
      
      prob = selected_probs[i].item()
      # add word combinations to branch_candidates in format [sentence, cumulative probability, all probs]
      branch_candidates.append((new_sent, candidate[1] * prob, candidate[2] + [prob]))
  
  return branch_candidates

def beam_gen(sent_tokens, candidates, depth=5, d='right', sample_size=2, topk=10, temperature=5):
  beams = candidates[:]
  new_candidates = candidates[:]
  while depth > 0:
    new_candidates = []
    for candidate in candidates:
      for new_candidate in candidates_gen(sent_tokens, candidate, d, sample_size, topk, temperature):
        beams.append(new_candidate)
        new_candidates.append(new_candidate)   
    print("Number of beams:", len(new_candidates))    
    candidates = new_candidates[:]
    depth -= 1
  sorted_beams = sorted(new_candidates, key=lambda tup: np.sum(np.log10(tup[2])), reverse=True)
  return beams, sorted_beams

def bi_generator(sent, direction, first_sample_size, sample_size, n_tokens, topk, iterations, temperature):
  sent_tokens = tokenizer.encode(sent, add_special_tokens=False) 

  for i in range(iterations):
    if (i % 2 == 0 and direction == 'both') or direction == 'left':
      print('>> left side generation')
      candidates = candidates_gen(sent_tokens=sent_tokens, d='left', n_candidates=first_sample_size,  topk=topk, temperature=temperature)
      beams, sorted_beams = beam_gen(sent_tokens, candidates, n_tokens-1, 'left', sample_size, topk, temperature=temperature)
      topn = len(sorted_beams)//5 if len(sorted_beams) > 4 else len(sorted_beams)
      selected_candidate = random.choice(sorted_beams[:topn])
      sent_tokens = selected_candidate[0] + sent_tokens
      print(tokenizer.decode(sent_tokens))
    if (i % 2 != 0 and direction == 'both') or direction == 'right':
      print('>> right side generation')
      candidates = candidates_gen(sent_tokens=sent_tokens, d='right', n_candidates=first_sample_size, topk=topk, temperature=temperature)
      beams, sorted_beams = beam_gen(sent_tokens, candidates, n_tokens-1, 'right', sample_size, topk, temperature=temperature)
      topn = len(sorted_beams)//5 if len(sorted_beams) > 4 else len(sorted_beams)
      selected_candidate = random.choice(sorted_beams[:topn])
      sent_tokens = sent_tokens + selected_candidate[0]
      print(tokenizer.decode(sent_tokens))
    
  return tokenizer.decode(sent_tokens)

sent = "Jesus called, he said he's sick of the disses"  
first_sample_size = 4
sample_size = 2
n_tokens = 4
topk = 20
iterations = 6
temperature = 4
direction = "both"

bi_generator(sent, direction, first_sample_size, sample_size, n_tokens, topk, iterations, temperature);


>> left side generation
Number of beams: 8
Number of beams: 16
Number of beams: 32
're dead But Jesus called, he said he's sick of the disses
>> right side generation
Number of beams: 8
Number of beams: 16
Number of beams: 32
're dead But Jesus called, he said he's sick of the disses What do he have
>> left side generation
Number of beams: 8
Number of beams: 16
Number of beams: 32
we all know we're dead But Jesus called, he said he's sick of the disses What do he have
>> right side generation
Number of beams: 8
Number of beams: 16
Number of beams: 32
we all know we're dead But Jesus called, he said he's sick of the disses What do he have no right to ask
>> left side generation
Number of beams: 8
Number of beams: 16
Number of beams: 32
me tacause we all know we're dead But Jesus called, he said he's sick of the disses What do he have no right to ask
>> right side generation
Number of beams: 8
Number of beams: 16
Number of beams: 32
me tacause we all know we're dead But Jesus called, he 

Approach 3: GPT-2

In [25]:
#Get the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

### Prepare data
df = pd.read_csv("/content/drive/MyDrive/adele.txt", sep="\n", header=None)

class SongLyrics(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in df['Lyric']:
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))               
        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)
        
    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]
    
dataset = df

In [26]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=5, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

model = train(dataset, model, tokenizer)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0
0



0it [00:00, ?it/s]


KeyError: ignored